# MWE of pathway analysis

# manually

Using Consensus Path DB-human to do pathway analysis at http://cpdb.molgen.mpg.de

Do over-representation analysis.
- provide foreground gene set (list of interesting genes for analysis)
- provide background gene set (optional, for our work we've used the concatonation of all the gene sets we're exploring as the background)
- run pathways as defined by pathway databases
- from enriched pathway-based sets we picked the top 3 pathways with qvalue < 0.05

# automated

Download web services interface from http://cpdb.molgen.mpg.de
Documentation available http://cpdb.molgen.mpg.de/download/CPDB.wsdl

Run code through python2
Needs module ZSI which needs inactive module XML (needs to be manually downloaded and installed)
 - ZSI has some socket issues when running the code but overall still faster when doing a pathway analysis for multiple gene sets
 


In [ ]:
#### PYTHON CODE TO RUN ####
############################

import random
import sys
import os
import pandas as pd

from cpdb_services import *
loc = cpdbLocator()
proxy = loc.getcpdb_portType(tracefile=sys.stdout)

main_dir = "pmdfiltered_top1000"
li = os.listdir(main_dir)

tot_bg = list()
for i in li:
    temp = pd.read_csv(main_dir+"/"+i, sep="\t", header=None)[0].to_list()
    tot_bg += temp
req = mapAccessionNumbersRequest()
req._accType = 'hgnc-symbol'
req._accNumbers = tot_bg
response = proxy.mapAccessionNumbers(req)
result = zip(response._accNumber, response._cpdbId)
bg = []
for r in result:
    if r[1]:
        bg.append(r[1].split(',')[0])

results_dir = "results"

while len(li) > 0:
    try:
        rli = os.listdir(results_dir)

        li = sorted(set(li) - set(rli))

        for file in li:
            fg = pd.read_csv(main_dir+"/"+file, sep="\t", header=None)[0].to_list()

            req = mapAccessionNumbersRequest()
            req._accType = 'hgnc-symbol' # 
            req._accNumbers = fg
            response = proxy.mapAccessionNumbers(req)
            result = zip(response._accNumber, response._cpdbId)
            fg = []
            for r in result:
                if r[1]:
                    fg.append(r[1].split(',')[0])

            req = overRepresentationAnalysisRequest()
            req._entityType = 'genes'
            req._fsetType = 'P'
            req._cpdbIdsFg = fg
            req._cpdbIdsBg = bg
            req._pThreshold = 1
            response = proxy.overRepresentationAnalysis(req)
            result = zip(response._name, response._details, response._overlappingEntitiesNum, response._allEntitiesNum, response._pValue, response._qValue)
            print result[:5]

            pd.DataFrame(result).to_csv("results"+"/"+file, sep="\t", header=False, index=False)
    except socket.error:
        pass